In [139]:
#parameters

root_path="C:\\Users\\bruno\\Desktop\\Diplomski\\ \\"

In [140]:
import pandas as pd
import numpy as np
import csv
import mne
from scipy.signal import find_peaks
from matplotlib import pyplot as plt, dates as mdates
import math
from datetime import datetime, timedelta, time
import pyarrow.feather as feather
from getEventTypes import getEventTypes

In [141]:
edf_path = root_path + "edf_signals.edf"
hypnogram_path = root_path + "csv_hypnogram.csv"
events_path = root_path + "csv_events.csv"

In [142]:
data = mne.io.read_raw_edf(edf_path, preload=True)

data_raw = data.get_data()
data_info = data.info
data_channels = data.ch_names

Extracting EDF parameters from C:\Users\bruno\Desktop\Diplomski\38011513_B\edf_signals.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 899999  =      0.000 ...  1799.998 secs...


C:\Users\bruno\AppData\Local\Temp\ipykernel_21192\1101522725.py:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'Flow Patient'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(edf_path, preload=True)


In [143]:
signal_no_sleep_no_events = pd.DataFrame(data_raw)

signal_no_sleep_no_events = signal_no_sleep_no_events.T
signal_no_sleep_no_events.columns = data_channels

yesterday = datetime.now() - timedelta(days=1)
start_time = data.info['meas_date'].replace(tzinfo=None).replace(year=yesterday.year, month=yesterday.month, day=yesterday.day)
end_time = start_time + timedelta(milliseconds=signal_no_sleep_no_events.shape[0]*2)

In [144]:
number_of_samples = signal_no_sleep_no_events.shape[0]
number_of_samples

900000

In [145]:
def shiftTimestampToYesterday(value):
    noon = time(12, 0)
    if(noon<value.time()):
        return value - timedelta(days=1)
    else:
        return value

In [174]:
hypno_df = pd.read_csv(hypnogram_path)
events_df = pd.read_csv(events_path, sep=';')

SleepPhases = {
    'WK' : 0,
    'N1' : 1,
    'N2' : 2,
    'N3' : 3,
    'REM' : 4}

hypno_df['Sleep Stage Int'] = hypno_df['Sleep Stage'].map(SleepPhases.get)
events_df['Time'] = pd.to_datetime(events_df['Time'])
events_df['Time'] = events_df['Time'].apply(shiftTimestampToYesterday) 

C:\Users\bruno\AppData\Local\Temp\ipykernel_21192\553362753.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  events_df['Time'] = pd.to_datetime(events_df['Time'])


In [175]:
hypno_df.rename(columns={' Epoch Number ':'Epoch','Sleep Stage Int': 'Stage','Sleep Stage': 'Stage Name' , 'Start Time ':'Start'}, inplace=True)
hypno_df['Start'] = pd.to_datetime(hypno_df['Start'])
hypno_df['Epoch'] = hypno_df['Epoch'].astype(int)
hypno_df['Start'] = hypno_df['Start'].apply(shiftTimestampToYesterday)
hypno_df = hypno_df.sort_values(by=['Start'])

C:\Users\bruno\AppData\Local\Temp\ipykernel_21192\1060567551.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  hypno_df['Start'] = pd.to_datetime(hypno_df['Start'])


In [176]:
events_df = events_df[(events_df['Time'] >= start_time) & (events_df['Time'] <= end_time)]
hypno_df = hypno_df[(hypno_df['Start'] >= start_time) & (hypno_df['Start'] <= end_time)]


In [177]:
continuous_time_series = pd.DataFrame(pd.date_range(start=start_time, periods = number_of_samples, freq='2ms'), columns=['timestamp'])
continuous_time_series

,timestamp
0,2023-08-19 22:00:18.000
1,2023-08-19 22:00:18.002
2,2023-08-19 22:00:18.004
3,2023-08-19 22:00:18.006
4,2023-08-19 22:00:18.008
...,...
899995,2023-08-19 22:30:17.990
899996,2023-08-19 22:30:17.992
899997,2023-08-19 22:30:17.994
899998,2023-08-19 22:30:17.996


In [178]:
events_df[events_df['Type']=='Snore']

,Type,Stage,Time,Epoch,Duration,Body Position,Validation
144,Snore,N2,2023-08-19 23:00:16,120,"7,0",Left,Not Validated


In [149]:
event_types = getEventTypes()

for event_type in event_types:
    event_type_rows = events_df[events_df['Type'] == event_type]

    for _, row in event_type_rows.iterrows():
        start_idx = int((row['Time'] - start_time).total_seconds() * 500)
        end_idx = start_idx + int(round(float(row['Duration'].replace(',','.')) * 500))
        continuous_time_series.loc[start_idx:end_idx, event_type] = True
        
    if len(event_type_rows)==0 :
        continuous_time_series[event_type] = False

continuous_time_series = continuous_time_series.fillna(False)

In [151]:
hypno_df.set_index('Start', inplace=True)
new_frequency = '2L'

hypno_df_resampled = hypno_df.resample(new_frequency)
hypno_df_resampled_interpolated = hypno_df_resampled.interpolate()
hypno_df_resampled_interpolated.reset_index(inplace=True)
hypno_df_resampled_interpolated['Epoch'] = hypno_df_resampled_interpolated['Epoch'].apply(round)
hypno_df_resampled_interpolated['Stage'] = hypno_df_resampled_interpolated['Stage'].apply(round)
hypno_df_resampled_interpolated

last_hypno_df_row = hypno_df_resampled_interpolated.iloc[-1]

In [152]:
hypno_df_resampled_interpolated = pd.concat([hypno_df_resampled_interpolated,pd.DataFrame([last_hypno_df_row] * (number_of_samples-len(hypno_df_resampled_interpolated)))], axis=0)

In [154]:
signal_to_filter = pd.merge(signal_no_sleep_no_events, hypno_df_resampled_interpolated, left_index=True, right_index=True)
signal_to_filter = pd.merge(signal_to_filter, continuous_time_series, left_index=True, right_index=True)
signal = signal_to_filter

Index(['EOG LOC-A2', 'EOG ROC-A2', 'EEG F3-A2', 'EEG F4-A1', 'EEG A1-A2',
       'EEG C3-A2', 'EEG C4-A1', 'EEG O1-A2', 'EEG O2-A1', 'EMG Chin',
       'Flow Patient-0', 'Flow Patient-1', 'Snore_x', 'Effort THO',
       'Effort ABD', 'SpO2', 'PulseRate', 'ECG I', 'Leg 1', 'Leg 2', 'Body',
       'Start', 'Epoch', 'Stage Name', 'Stage', 'timestamp',
       'Absolute Desaturation', 'Arousal', 'Bradycardia', 'Central Apnea',
       'Hypopnea', 'Leg Movement', 'Long RR', 'Mixed Apnea',
       'Obstructive Apnea', 'PTT Drop', 'Periodic Respiration',
       'Relative Desaturation', 'Snore_y', 'Tachycardia'],
      dtype='object')

In [155]:
feather.write_feather(signal, root_path + "signal.feather")
%time

CPU times: total: 0 ns
Wall time: 0 ns


In [156]:
saturation = signal['SpO2']

In [157]:
x = saturation
peaks2, _ = find_peaks(x, prominence=2)      # BEST!
local_minimum_indices, _ = find_peaks(-x, prominence=2)
local_maximum_indices, _ = find_peaks(x, prominence=2)

In [158]:
local_minimum_values = x[local_minimum_indices]
local_maximum_values = x[local_maximum_indices]
differences = []
for min_val, min_idx in zip(local_minimum_values, local_minimum_indices):
    for max_val, max_idx in zip(local_maximum_values, local_maximum_indices):
        if max_idx > min_idx:
            differences.append(max_val - min_val)

In [159]:
df = pd.DataFrame(differences)

In [ ]:
local_minimum_values = signal.loc[local_minimum_indices]
local_maximum_values = signal.loc[local_maximum_indices]
connected_pairs = []


for i, min_idx in enumerate(local_minimum_indices):
    if(len(local_maximum_indices[local_maximum_indices < min_idx])>0):
        max_idx = local_maximum_indices[local_maximum_indices < min_idx][-1]
        connected_pairs.append((max_idx, min_idx))

differences = []
for max_idx, min_idx in connected_pairs:
    max_val = x[max_idx]
    min_val = x[min_idx]
    diff = min_idx - max_idx
    differences.append((diff, max_idx, min_idx, max_val, min_val))

df = pd.DataFrame(differences, columns=['Index Difference', 'Maximum Index', 'Minimum Index', 'Maximum Value', 'Minimum Value'])
df

In [161]:
df2 = df

In [162]:
df2 = df.loc[(df['Index Difference'] >= 5000) & (df['Index Difference'] <= 40000)]

In [163]:
diffDf = df2['Maximum Value'] - df2['Minimum Value']
df2 = pd.concat([df2, diffDf], axis=1)
df2.columns = [*df2.columns[:-1], 'diff']

In [164]:
df3 = df2.loc[(df2['diff'] >= 3)]

In [165]:
zero_values = df3.loc[(df3['Minimum Value'] == 0)]

In [166]:
df4 = df3.drop(zero_values.index)

In [167]:
saturation = saturation[saturation!= 0]

In [168]:
df4['Index Difference sek'] = df4['Index Difference']/1000

In [ ]:
df4

In [170]:
def direction_coefficient(row):
    dy = row['y2'] - row['y1']
    dx = row['x2'] - row['x1']
    return (row['y2'] - row['y1']) / (row['x2'] - row['x1'])


df = pd.DataFrame({'x1': df4['Maximum Index'],
                   'y1': df4['Maximum Value'],
                   'x2': df4['Minimum Index'],
                   'y2': df4['Minimum Value']})

df['slope'] = df.apply(direction_coefficient, axis=1)
df['angle_degrees'] = df['slope'].apply(lambda x: math.degrees(x))

In [171]:
df_slope = pd.merge(df4, df, left_index=True, right_index=True)

In [172]:
feather.write_feather(df_slope, root_path + "slope.feather")